In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
data = pd.read_csv('/content/d1.csv')
data.columns = data.columns.str.strip()

features = [
    'Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Flow Bytes/s',
    'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
    'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
    'Active Mean', 'Idle Mean', 'Active Max', 'Idle Max'
]
label = 'Label'

X = data[features].values
y = data[label].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e9, neginf=-1e9)
y_train = np.nan_to_num(y_train, nan=0.0, posinf=1e9, neginf=-1e9)
X_test = np.nan_to_num(X_test, nan=0.0, posinf=1e9, neginf=-1e9)
y_test = np.nan_to_num(y_test, nan=0.0, posinf=1e9, neginf=-1e9)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

def build_model1(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation='relu', return_sequences=True, input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.LSTM(64, activation='relu', return_sequences=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mse')
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
model1 = build_model1(input_shape)

history = model1.fit(
    X_train, y_train,
    epochs=3, batch_size=128,
    validation_data=(X_test, y_test)
)

predictions = model1.predict(X_train[:5])
print("Sample predictions:", predictions)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
19606/19606 ━━━━━━━━━━━━━━━━━━━━ 375s 19ms/step - loss: 0.2837 - val_loss: 0.9502
Epoch 2/3
19606/19606 ━━━━━━━━━━━━━━━━━━━━ 376s 18ms/step - loss: 0.0157 - val_loss: 0.4717
Epoch 3/3
19606/19606 ━━━━━━━━━━━━━━━━━━━━ 390s 19ms/step - loss: 0.0061 - val_loss: 0.0990
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step
Sample predictions: [[0.94535565]
 [0.9824203 ]
 [0.96468335]
 [0.97886354]
 [0.94231296]]


In [ ]:
y_pred_prob = model1.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

print("Confusion Matrix:")
print(conf_matrix)
print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("True Positives:", tp)

print(classification_report(y_test, y_pred))

19606/19606 ━━━━━━━━━━━━━━━━━━━━ 67s 3ms/step
Confusion Matrix:
[[   291    122]
 [   148 626800]]
True Negatives: 291
False Positives: 122
False Negatives: 148
True Positives: 626800
              precision    recall  f1-score   support

           0       0.66      0.70      0.68       413
           1       1.00      1.00      1.00    626948

    accuracy                           1.00    627361
   macro avg       0.83      0.85      0.84    627361
weighted avg       1.00      1.00      1.00    627361



In [ ]:
model1.save('model1.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
